In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2

In [ ]:
%%time
import decimal


def pi():
    """
    Compute Pi to the current precision.

    Examples
    --------
    >>> print(pi())
    3.141592653589793238462643383

    Notes
    -----
    Taken from https://docs.python.org/3/library/decimal.html#recipes
    """
    decimal.getcontext().prec += 2  # extra digits for intermediate steps
    three = decimal.Decimal(3)      # substitute "three=3.0" for regular floats
    lasts, t, s, n, na, d, da = 0, three, 3, 1, 0, 0, 24
    while s != lasts:
        lasts = s
        n, na = n + na, na + 8
        d, da = d + da, da + 32
        t = (t * n) / d
        s += t
    decimal.getcontext().prec -= 2
    return +s               # unary plus applies the new precision

decimal.getcontext().prec = 100000
pi = pi()

In [ ]:
import requests

In [ ]:
url = 'http://introcs.cs.princeton.edu/data/pi-10million.txt'

r = requests.get(url)

pi_string = r.text

n_digits = len(pi_string)

pi_digits = np.zeros(n_digits)

for i, dig in tqdm(enumerate(pi_string)):
    pi_digits[i] = int(dig)

pi_digits = pi_digits.astype(int)

pi_stack = np.stack([pi_digits, np.roll(pi_digits, -1), np.roll(pi_digits, -2)])

pidf = pd.DataFrame(pi_stack.T.astype(int))

In [ ]:
pidf['is_1'] = pidf[0] == 1
pidf['is_7'] = pidf[0] == 7
pidf['is_11'] = pidf['is_1'] & (pidf[1] == 1)
pidf['is_17'] = pidf['is_1'] & (pidf[1] == 7)
pidf['is_117'] = pidf['is_11'] & (pidf[2] == 7)

In [ ]:
mat11 = pidf['is_11'].values.reshape((4000, 2500))
mat17 = pidf['is_17'].values.reshape((4000, 2500))
mat117 = pidf['is_117'].values.reshape((4000, 2500))

In [ ]:
xs, ys = np.nonzero(mat11)
p11s = MultiPoint([Point(x,y) for x,y in zip(xs, ys)])
c11s = MultiLineString([p.buffer(2.5, resolution=2).boundary for p in p11s])    

In [ ]:
xs, ys = np.nonzero(mat17)
p17s = MultiPoint([Point(x,y) for x,y in zip(xs, ys)])
c17s = MultiLineString([p.buffer(2.5, resolution=2).boundary for p in p17s])    

In [ ]:
page_x_inches: float = 17. # inches
page_y_inches: float = 11 # inches
border:float = 0.



px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds


In [ ]:
drawbox = drawbox.buffer(-20)

In [ ]:
xs, ys = np.nonzero(mat117)
p117s = MultiPoint([Point(x,y) for x,y in zip(xs, ys)])
c117s = MultiLineString([p.buffer(10, resolution=4).boundary for p in p117s])    

In [ ]:
sc117s = gp.center_at(gp.scale_like(c117s, drawbox), drawbox.centroid)

In [ ]:
tris = so.triangulate(p117s, edges=True)

In [ ]:
sc_tris = gp.center_at(gp.scale_like(MultiLineString(tris), drawbox), drawbox.centroid)

In [ ]:
sc_tris

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)

sk.scale('1mm')
sk.stroke(1)
sk.geometry(sc_tris)
# sk.stroke(2)
# sk.geometry(c11s)
# sk.stroke(3)
# sk.geometry(c17s)
sk.display(color_mode='layer')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0104_117_lattice.svg'

sk.save(savepath)

In [ ]:
vpype_commands = 'reloop splitall linesimplify --tolerance 0.2mm linemerge --tolerance 0.2mm linesort'
vpype_str = f'vpype read -q 0.1mm {savepath} {vpype_commands} write {savepath}'

os.system(vpype_str)

In [ ]:
page_x_inches: float = 14. # inches
page_y_inches: float = 11 # inches
border:float = 20.



px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds

In [ ]:
def calc_r(theta, phi):
    return phi ** (theta * 2 / np.pi)

In [ ]:
f = functools.partial(calc_r, phi=1.005)

In [ ]:
thetas = np.linspace(0, np.pi*100, n_digits)
r = f(thetas) - 0.99

In [ ]:
xs = np.cos(thetas) * r
ys = np.sin(thetas) * r

In [ ]:
where_117 = np.nonzero(pidf['is_117'].values)[0]

In [ ]:
x117s = xs[pidf['is_117']]
y117s = ys[pidf['is_117']]

In [ ]:
spiral_line = np.stack([xs, ys]).T

In [ ]:
dashes = []
for i in tqdm(range(0, len(where_117)-1, 2)):
    start = where_117[i]
    stop = where_117[i+1]
    dashes.append(spiral_line[start:stop, :])

In [ ]:
mls = MultiLineString([LineString(d) for d in dashes])

In [ ]:
smls = gp.center_at(gp.scale_like(mls, drawbox), drawbox.centroid)

In [ ]:
bmls = [p.buffer(np.random.uniform(0.5, 18.5), resolution=1, cap_style=2, join_style=2) for p in smls]
bmls = so.unary_union(MultiPolygon(bmls)).boundary
bmls = gp.center_at(gp.scale_like(bmls, drawbox), drawbox.centroid)

In [ ]:
ub = [p.buffer(1, resolution=1, cap_style=2, join_style=2) for p in smls]
ub = so.unary_union(MultiPolygon(ub)).boundary
ub = gp.center_at(gp.scale_like(ub, drawbox), drawbox.centroid)

In [ ]:
bmls = [p.buffer(np.random.uniform(0.1, 0.5), resolution=1, cap_style=2, join_style=2) for p in bmls]
bmls = so.unary_union(MultiPolygon(bmls)).boundary
bmls = gp.center_at(gp.scale_like(bmls, drawbox), drawbox.centroid)

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)

sk.scale('1mm')
sk.stroke(2)
sk.geometry(smls)
# sk.stroke(3)
# sk.geometry(bmls)
for t in [0.3, 0.3, 0.3, 0.4, 0.5, 1, 2, 3,]:
    print(t)
    sk.vpype(f'linemerge --tolerance {t}')

for t in [0.3, 0.3, 0.3, 0.4, 0.5, ]:
    print(t)
    sk.vpype(f'linesimplify --tolerance {t}')

    
sk.display(color_mode='layer')
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0178_spiral.svg'

sk.save(savepath)

In [ ]:

p117s = MultiPoint([Point(x,y) for x,y in zip(x117s, y117s)])  
sp117s = gp.center_at(gp.scale_like(p117s, drawbox), drawbox.centroid)

In [ ]:
bs = [p.buffer(np.random.uniform(0.2, 1.5), resolution=1) for p in sp117s]
bs = so.unary_union(MultiPolygon(bs)).boundary